In [ ]:
import sqlite3
import numpy as np
import pandas as pd
# import seaborn as sb
import matplotlib.pyplot as plt

In [ ]:
plt.rcParams['font.family'] = 'Palatino'
fig_path = r'/Users/leonbecker/Library/CloudStorage/OneDrive-UniversitätWürzburg/Universität Würzburg/Master Information-Systems/Thesis/LaTex/graphics'

In [ ]:
wine_type_ids = {
    1: 'red',
    2: 'white',
    3: 'sparkling_wine',
    4: 'rose',
    24: 'liqueur_wine',
    7: 'dessert_wine'
}

In [ ]:
con = sqlite3.connect('wine_distinct.db')
cur = con.cursor()
df_wine = pd.read_sql_query("SELECT * from wine_distinct", con)
df_wine.head()

In [ ]:
for index in df_wine.index:
    df_wine.at[index, 'type_name'] = wine_type_ids[df_wine.loc[index, 'type_id']]

In [ ]:
ax = df_wine.country.value_counts().head(n=10).plot(
    kind='bar',
    color='black',
    zorder=3
)
plt.grid(zorder=0, axis='y')
plt.xticks(rotation=45)
plt.tight_layout()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.savefig(fig_path + '/top_10_producer_countries.pdf')

In [ ]:
ax = df_wine.type_name.value_counts().plot(
    kind='bar',
    color='black',
    zorder=3
)
plt.grid(zorder=0, axis='y')
plt.xticks(rotation=0)
plt.tight_layout()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.savefig(fig_path + '/wine_type_distribution.pdf')

In [ ]:
ax = df_wine.average_rating.plot(
    kind='box'
)
plt.grid(axis='y')
plt.tight_layout()
plt.xticks(rotation=0)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.savefig(fig_path + '/average_rating.pdf')

In [ ]:
ax = df_wine.boxplot(by='type_name', column='average_rating', rot=45)
ax.set_title('')
plt.suptitle('')
ax.set_xlabel('')
plt.tight_layout()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.savefig(fig_path + '/average_rating_by_wine.pdf')

In [ ]:
top_10_countries = df_wine.country.value_counts().head(n=10).index

In [ ]:
plt.grid(zorder=0)
ax = sb.violinplot(
    data=df_wine[df_wine.country.isin(top_10_countries)],
    y='average_rating',
    x='country',
    #palette=['w','w','w','w', 'w','w','w','w', 'w', 'w'],
    zorder=3
)
# plt.grid(axis='y')
plt.xticks(rotation=45)
plt.ylabel(None)
plt.xlabel(None)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.tight_layout()
plt.savefig(fig_path + '/top_10_ratings_for_countries.pdf')

In [ ]:
con_rev = sqlite3.connect('review.db')
df_rev = pd.read_sql_query("SELECT * from review", con_rev)
df_rev.head()

In [ ]:
ax = df_rev.note_length.plot(
    kind='box'
)
plt.grid(axis='y')
plt.tight_layout()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
ax.set_xticks([])
plt.savefig(fig_path + '/note_length.pdf')

In [ ]:
df_rev.note_length.plot(
    kind='box'
)
plt.ylabel(None)
plt.xlabel(None)
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.tight_layout()

In [ ]:
ax = df_rev.len_code.value_counts().head(n=10).plot(
    kind='bar',
    color='black',
    zorder=3
)
plt.grid(axis='y')
plt.tight_layout()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.xticks(rotation=0)
# ax.set_xticks([])
plt.savefig(fig_path + '/language_distribution.pdf')

In [ ]:
con_rev_en = sqlite3.connect('review_en.db')
df_rev_en = pd.read_sql_query("SELECT * from review_en", con_rev_en)
df_rev_en.head()

In [ ]:
ax = df_rev_en.note_length.plot(
    kind='box',
    # color='black',
    #zorder=3
)
plt.grid(axis='y')
plt.tight_layout()
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)
plt.xticks(rotation=0)
ax.set_xticks([])
plt.savefig(fig_path + '/note_length_en.pdf')